This code was used to get real time temp, snowfall, and depth (TBD what to do with these stats). 

The elevs, colors, lifts, and prices function is now in make_mtn_df.py.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
import pickle

In [2]:
browser = webdriver.PhantomJS()

In [3]:
resort_urls = {'Loveland': 'colorado/loveland',
               'Arapahoe Basin': 'colorado/arapahoe-basin-ski-area',
               'Copper': 'colorado/copper-mountain-resort',
               'Eldora': 'colorado/eldora-mountain-resort',
               'Alpine Meadows': 'california/squaw-valley-usa',
               'Vail': 'colorado/vail',
               'Monarch': 'colorado/monarch-mountain',
               'Crested Butte': 'colorado/crested-butte-mountain-resort',
               'Taos': 'new-mexico/taos-ski-valley',
               'Diamond Peak': 'nevada/diamond-peak',
               'Winter Park': 'colorado/winter-park-resort',
               'Beaver Creek': 'colorado/beaver-creek'}

In [53]:
URL = 'http://www.onthesnow.com/'+resort_urls['Winter Park']+'/ski-resort.html'
browser.get(URL)
# time.sleep(3)

In [54]:
soup = BeautifulSoup(browser.page_source,'html.parser')
temp_rows = soup.select('p.bluetxt.temp') 
snowfall_rows = soup.select('p.bluetxt.sfa')
sdepth_rows = soup.select('p.bluetxt.sd') 

In [89]:
temp = int(''.join([x for x in [cell.text for cell in temp_rows][0] if x.isnumeric()]))
snowfall = int(''.join([x for x in [cell.text for cell in snowfall_rows][0] if x.isnumeric()]))
depth = int(''.join([x if x.isnumeric() else '0' for x in [cell.text for cell in sdepth_rows][0]]))

In [90]:
temp, snowfall, depth

(53, 0, 0)

In [91]:
def get_stats(resort):
    URL = 'http://www.onthesnow.com/'+resort_urls[resort]+'/ski-resort.html'
    browser.get(URL)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    temp_rows = soup.select('p.bluetxt.temp') 
    snowfall_rows = soup.select('p.bluetxt.sfa')
    sdepth_rows = soup.select('p.bluetxt.sd') 
    temp = int(''.join([x for x in [cell.text for cell in temp_rows][0] if x.isnumeric()]))
    snowfall = int(''.join([x for x in [cell.text for cell in snowfall_rows][0] if x.isnumeric()]))
    depth = int(''.join([x if x.isnumeric() else '0' for x in [cell.text for cell in sdepth_rows][0]]))
    return temp, snowfall, depth

In [92]:
get_stats('Winter Park')

(53, 0, 0)

In [93]:
for resort in resort_urls:
    print(get_stats(resort))

(47, 0, 0)
(47, 0, 0)
(49, 0, 0)
(57, 0, 0)
(37, 0, 0)
(52, 0, 0)
(48, 0, 0)
(44, 0, 0)
(53, 0, 0)
(36, 0, 0)
(53, 0, 0)
(53, 0, 0)


In [55]:
stuff = soup.select('table.ovv_info tbody tr td')

In [56]:
rows = [cell.text for cell in stuff]
rows

['\nClosed\nSeason Start/End: 11/15 - 4/29\n',
 '9000ft - 12060ft',
 '\n24%|\n                38%|\n                34%|\n                5%\n',
 '25',
 '\n                N/A\n            ']

In [59]:
_, elevs, colors, lifts, price = rows

In [10]:
bottom = int(''.join([x for x in elevs.split()[0] if x.isnumeric()]))
top = int(''.join([x for x in elevs.split()[2] if x.isnumeric()]))
bottom, top

(10800, 13010)

In [11]:
greens = int(''.join([x for x in colors.split()[0] if x.isnumeric()]))
blues = int(''.join([x for x in colors.split()[1] if x.isnumeric()]))
blacks = int(''.join([x for x in colors.split()[2] if x.isnumeric()]))
bbs = int(''.join([x for x in colors.split()[3] if x.isnumeric()]))
greens, blues, blacks, bbs

(13, 41, 20, 26)

In [12]:
lifts = int(lifts)
lifts

10

In [61]:
price.split()

['N/A']

In [62]:
price_split = float([x for x in price.split() if x.startswith('US')][1][3:])
price_split

IndexError: list index out of range

In [145]:
def get_elevs_colors_lifts_price(resort):
    URL = 'http://www.onthesnow.com/'+resort_urls[resort]+'/ski-resort.html'
    browser.get(URL)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    stuff = soup.select('table.ovv_info tbody tr td')
    rows = [cell.text for cell in stuff]
    _, elevs, colors, lifts, price = rows
    bottom = int(''.join([x for x in elevs.split()[0] if x.isnumeric()]))
    top = int(''.join([x for x in elevs.split()[2] if x.isnumeric()]))
    greens = int(''.join([x for x in colors.split()[0] if x.isnumeric()]))
    blues = int(''.join([x for x in colors.split()[1] if x.isnumeric()]))
    blacks = int(''.join([x for x in colors.split()[2] if x.isnumeric()]))
    bbs = int(''.join([x for x in colors.split()[3] if x.isnumeric()]))
    lifts = int(lifts)
    price = int(''.join([x if x.isnumeric() else '0' for x in price.split()[0]]))
    return [bottom, top, greens, blues, blacks, bbs, lifts, price]

In [147]:
elevs_colors_lifts_price = {}
for resort in resort_urls:
    elevs_colors_lifts_price[resort] = get_elevs_colors_lifts_price(resort)
    print(elevs_colors_lifts_price[resort])

(10800, 13010, 13, 41, 20, 26, 10, 0)
(10779, 13050, 10, 30, 37, 23, 8, 0)
(9712, 12312, 21, 25, 36, 18, 23, 0)
(9200, 10800, 21, 51, 19, 9, 11, 0)
(6200, 9050, 25, 42, 33, 0, 42, 0)
(8120, 11570, 18, 29, 28, 25, 31, 0)
(10789, 11952, 23, 28, 36, 13, 6, 0)
(9100, 12162, 26, 57, 14, 3, 15, 0)
(9206, 12481, 24, 25, 33, 18, 16, 0)
(6700, 8539, 10, 42, 48, 0, 7, 0)
(9000, 12060, 24, 38, 34, 5, 25, 0)
(8100, 11439, 19, 42, 22, 17, 25, 0)
